In [45]:
import json
file = r'train.json'
with open(file) as train_file:
    dict_train = json.load(train_file)

In [46]:
print(dict_train[0])

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}


In [47]:
def stringify(list_of_words):
  ing_string = ''
  space = ' '
  for word in list_of_words:
    ing_string = ing_string + space +  word 
  return ing_string

In [48]:
#Construct dataset
X = []
y = []
ids = []
for i in range(len(dict_train)):
  ids.append(dict_train[i]['id'])
  X.append(stringify(dict_train[i]['ingredients']))
  y.append(dict_train[i]['cuisine'])

In [49]:
import pandas as pd
df = pd.DataFrame({'id':ids, 
                   'cuisine':y, 
                   'ingredients':X})
print(df.head(5))

      id      cuisine                                        ingredients
0  10259        greek   romaine lettuce black olives grape tomatoes g...
1  25693  southern_us   plain flour ground pepper salt tomatoes groun...
2  20130     filipino   eggs pepper salt mayonaise cooking oil green ...
3  22213       indian                     water vegetable oil wheat salt
4  13162       indian   black pepper shallots cornflour cayenne peppe...


In [66]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer(num_words = 1500, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(list(df['ingredients']))
X = tokenizer.texts_to_matrix(df['ingredients'])
X = pad_sequences(X)
Y = pd.get_dummies(df['cuisine']).values

In [83]:
index = tokenizer.word_index
words = []
nb = 0
for word in index:
  words.append(word)

In [86]:
word_dict = {}
for word in words[:1500]:
  word_dict[word] = index.get(word)-1 

In [95]:
# Make the embedding 
from keras.models import Input, Model
from keras.layers import Dense

# Defining the size of the embedding
embed_size = 20

# Defining the neural network
inp = Input(shape=(X.shape[1]))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Optimizing the network weights
model.fit(x=X, y=Y, batch_size=256, epochs = 50)

# Obtaining the weights from the neural network. 
# These are the so called word embeddings

# The input layer 
weights = model.get_weights()[0]
model

# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[word_dict.get(word)]
        })

Epoch 1/50
156/156 [==============================] - 1s 4ms/step - loss: 1.9653
Epoch 2/50
156/156 [==============================] - 1s 4ms/step - loss: 1.1724
Epoch 3/50
156/156 [==============================] - 1s 4ms/step - loss: 0.9368
Epoch 4/50
156/156 [==============================] - 1s 4ms/step - loss: 0.8224
Epoch 5/50
156/156 [==============================] - 1s 4ms/step - loss: 0.7530
Epoch 6/50
156/156 [==============================] - 1s 4ms/step - loss: 0.7062
Epoch 7/50
156/156 [==============================] - 1s 4ms/step - loss: 0.6727
Epoch 8/50
156/156 [==============================] - 1s 4ms/step - loss: 0.6466
Epoch 9/50
156/156 [==============================] - 1s 4ms/step - loss: 0.6259
Epoch 10/50
156/156 [==============================] - 1s 4ms/step - loss: 0.6080
Epoch 11/50
156/156 [==============================] - 1s 4ms/step - loss: 0.5933
Epoch 12/50
156/156 [==============================] - 1s 5ms/step - loss: 0.5801
Epoch 13/50
156/156 [====

In [105]:
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[word_dict.get(word)]
        })

In [107]:
#Save Example
np.save('embedding.npy',embedding_dict)

In [109]:
#Recover Example
d2=np.load('embedding.npy', allow_pickle=True)
print(d2.item().get('pepper'))

[0.005949706, 0.05450415, -0.002995953, 0.02902805, -0.005515933, -0.052571464, 0.010851413, -0.06108242, 0.05341339, 0.055890366, -0.053286728, -0.029714692, -0.008330889, -0.023648415, -0.003532108, 0.05428815, 0.022221483, 0.040627234, 0.029265307, -0.04991974]
